# Example for steel production, equivalent to AMPL `steelT.mod`, using CPLEX
## Copyright (C) Princeton Consultants, 2017-2018
### First, import the pandas library

In [1]:
import pandas as pd

import sys
sys.path.append("C:/EclipseWorkspaces/LiClipseWorkspace/OptiPandas/src")
import optipandas as opd
opd.init('CPLEX')


### Read the data associated with the products

In [3]:
productDF = pd.read_excel("steelT.xlsx", index_col=0, skip_footer=18)
productDF

,rate,inv0,prodcost,invcost
Product,,,,
bands,220,10,10,2.5
coils,154,0,11,3.0


### Read the revenue and market data.  Note that we change the name of the index, because the Excel file has the name of the table, which is interpreted as the name of the index

In [4]:
revenue = pd.read_excel("steelT.xlsx", index_col=0, skiprows=6, skip_footer=12)
revenue.index.name = 'Product'
revenue

,1,2,3,4
Product,,,,
bands,25,26,27,27
coils,30,35,37,39


In [5]:
market = pd.read_excel("steelT.xlsx", index_col=0, skiprows=10, skip_footer=8)
market.index.name = 'Product'
market

,1,2,3,4
Product,,,,
bands,6000,6000,4000,6500
coils,4000,2500,3500,4200


### Read the available capacity per time period

In [6]:
avail = pd.read_excel("steelT.xlsx", index_col=0, skiprows=16, usecols=1).avail
avail

Time
1    40
2    40
3    32
4    40
Name: avail, dtype: int64

### Compute a tidy version of the revenue and market data

In [7]:
rmDF = pd.concat(
    [market.rename_axis("T", axis="columns").stack().rename("market"),
     revenue.rename_axis("T", axis="columns").stack().rename("revenue")],
    axis=1)
rmDF

market  revenue
Product T                 
bands   1    6000       25
        2    6000       26
        3    4000       27
        4    6500       27
coils   1    4000       30
        2    2500       35
        3    3500       37
        4    4200       39

### Import the DoCplex library

In [10]:
from docplex.mp.model import Model

### Create a DoCplex modeling object

In [11]:
model = Model(name='steelT')

### Create a variety of Series containing the decision variables. Note that for the Sell variables, we can get the upper bounds right from the table. For the Inv variables, we have to create an index that includes time period 0

In [13]:
Make = pd.Series(model.continuous_var_list(rmDF.index, name='Make'), index=rmDF.index, name="Make")
Sell = pd.Series(model.continuous_var_list(rmDF.index, ub=list(rmDF.market.values), name='Sell'),
                 index=rmDF.index, name="Sell")
invindex = rmDF.index.union(pd.MultiIndex.from_product([rmDF.index.get_level_values('Product'),[0]], names=['Product','T']))
Inv = pd.Series(model.continuous_var_list(invindex, name='Inv'), index=invindex, name="Inv")

pd.concat([Make, Sell, Inv], axis=1)

Make               Sell               Inv
Product T                                                        
bands   0                NaN                NaN  Inv_('bands', 0)
        1  Make_('bands', 1)  Sell_('bands', 1)  Inv_('bands', 1)
        2  Make_('bands', 2)  Sell_('bands', 2)  Inv_('bands', 2)
        3  Make_('bands', 3)  Sell_('bands', 3)  Inv_('bands', 3)
        4  Make_('bands', 4)  Sell_('bands', 4)  Inv_('bands', 4)
coils   0                NaN                NaN  Inv_('coils', 0)
        1  Make_('coils', 1)  Sell_('coils', 1)  Inv_('coils', 1)
        2  Make_('coils', 2)  Sell_('coils', 2)  Inv_('coils', 2)
        3  Make_('coils', 3)  Sell_('coils', 3)  Inv_('coils', 3)
        4  Make_('coils', 4)  Sell_('coils', 4)  Inv_('coils', 4)

### Note that the types of the columns are objects.  In fact, they are decision variables

In [14]:
(Make.dtype, Sell.dtype, Inv.dtype), type(Make.iloc[0])

((dtype('O'), dtype('O'), dtype('O')), docplex.mp.linear.Var)

### Set the objective function as
$$ \hbox{maximize}\quad\sum_{p,t}(revenue[p,t]*Sell[p,t]-prodcost[p]*Make[p,t]-invcost[p]*Inv[p,t])$$

In [15]:
model.maximize(model.sum(rmDF.revenue*Sell-productDF.prodcost*Make-productDF.invcost*Inv[rmDF.index]))

### State the constraints limiting production
$$\sum_p Make[p,t]*(1.0/rate[p]) \le avail[t], \qquad\forall t$$

In [24]:
avail

Time
1    40
2    40
3    32
4    40
Name: avail, dtype: int64

In [23]:
opd.sum("Product", Make/productDF.rate) <= avail

T
1    0.005Make_('bands', 1)+0.006Make_('coils', 1) ...
2    0.005Make_('bands', 2)+0.006Make_('coils', 2) ...
3    0.005Make_('bands', 3)+0.006Make_('coils', 3) ...
4    0.005Make_('bands', 4)+0.006Make_('coils', 4) ...
dtype: object

In [12]:
opd.forall(model, "T", (opd.sum("Product", Make/productDF.rate) <= avail), "Time")

T
1    Time[1]: 0.005Make_('bands', 1)+0.006Make_('co...
2    Time[2]: 0.005Make_('bands', 2)+0.006Make_('co...
3    Time[3]: 0.005Make_('bands', 3)+0.006Make_('co...
4    Time[4]: 0.005Make_('bands', 4)+0.006Make_('co...
Name: Time, dtype: object

### Use slicing to pick up the Inventory variables for period 0

In [16]:
idx = pd.IndexSlice
Inv.sort_index(inplace=True)
Inv[idx[:,0]]

Product
bands    Inv_('bands', 0)
coils    Inv_('coils', 0)
Name: Inv, dtype: object

### Add the constraint for the first time period
$$Inv[p,0] = inv0[p] \qquad\forall p$$

In [14]:
opd.forall(model, "Product", (Inv[idx[:,0]] == productDF.inv0), "Init_Inv")

Product
bands    Init_Inv[bands]: Inv_('bands', 0) == 10
coils     Init_Inv[coils]: Inv_('coils', 0) == 0
Name: Init_Inv, dtype: object

### Now add the balance constraints. Note the use of `.shift` to refer to the earlier period
$$Make[p,1] + Inv[p,t-1] = Sell[p,1]+Inv[p,1]\qquad\forall p, t\ge 1$$

In [17]:
opd.forall(model, ["Product", "T"], (Make + Inv.groupby('Product').shift(1)[rmDF.index] == Sell + Inv[rmDF.index]), "Balance")


Product  T
bands    1    Balance[bands,1]: Make_('bands', 1)+Inv_('band...
         2    Balance[bands,2]: Make_('bands', 2)+Inv_('band...
         3    Balance[bands,3]: Make_('bands', 3)+Inv_('band...
         4    Balance[bands,4]: Make_('bands', 4)+Inv_('band...
coils    1    Balance[coils,1]: Make_('coils', 1)+Inv_('coil...
         2    Balance[coils,2]: Make_('coils', 2)+Inv_('coil...
         3    Balance[coils,3]: Make_('coils', 3)+Inv_('coil...
         4    Balance[coils,4]: Make_('coils', 4)+Inv_('coil...
Name: Balance, dtype: object

In [19]:
Inv.groupby('Product').shift(1).to_frame('ShiftInv')

ShiftInv
Product T                  
bands   0               NaN
        1  Inv_('bands', 0)
        2  Inv_('bands', 1)
        3  Inv_('bands', 2)
        4  Inv_('bands', 3)
coils   0               NaN
        1  Inv_('coils', 0)
        2  Inv_('coils', 1)
        3  Inv_('coils', 2)
        4  Inv_('coils', 3)

### Time to solve

In [20]:
model.solve()

docplex.mp.solution.SolveSolution(obj=940250,values={Sell_('bands', 1):6..

### Grab the solution and put the solution in a DataFrame

In [17]:
soln = pd.DataFrame({'Make' : [x.solution_value for x in Make],
                     'Sell' : [x.solution_value for x in Sell],
                     'Inv' : [x.solution_value for x in Inv[rmDF.index]]},
                    index = rmDF.index)
soln

Inv    Make    Sell
Product T                       
bands   1    0.0  5990.0  6000.0
        2    0.0  6000.0  6000.0
        3    0.0  2040.0  2040.0
        4    0.0  2800.0  2800.0
coils   1  540.0  1967.0  1427.0
        2    0.0  1960.0  2500.0
        3    0.0  3500.0  3500.0
        4    0.0  4200.0  4200.0